# Weather Modeling

## Introduction

In this lecture, we will learn how to model weather data. We will use the data from the [National Oceanic and Atmospheric Administration (NOAA) Global Historical Climatology Network](https://www.ncdc.noaa.gov/data-access/land-based-station-data/land-based-datasets/global-historical-climatology-network-ghcn) (GHCN) to model the temperature in the United States. The GHCN is a database of daily climate records from thousands of land surface stations across the globe.

## Warning {.hide}

::: {.callout-warning}
Many simplifications have been made in order to complete this demonstration in the lecture period.  There are a number of things that could be done to improve how the data is retrieved and modeled.  It is intended to demonstrate functionality on how a Python application can be built to model weather data.
:::


## API Token {.hide .smaller-85}

To download the data, you will need to sign up for an API token on the NOAA website.  The National Climatic Data Center (NCDC) provides access to the GHCN data through Climate Data Online (CDO) web services. The API token is used to authenticate your requests to the CDO web services. You can sign up for an API token [here](https://www.ncdc.noaa.gov/cdo-web/token).

![](./token-request.png)

## Python Packages

We will need the following Python packages to download and analyze the weather data:

- `requests`: to make HTTP requests to the NOAA API
- `pandas`: to work with tabular data
- `matplotlib`: to create plots
- `scipy`: to fit a model to the data
- `python-dotenv`: to load the API token from the `.env` file


## `requirements.txt` {.smaller-80}

A requirements file is a simple text file that contains a list of Python packages that are required for the project. You can create a `requirements.txt` file in the same directory as this notebook with the following contents:

```
requests
pandas
matplotlib
python-dotenv
prophet
jupyter
ipywidgets
plotly
```

You can install the required packages using the following command:

```
pip install -r requirements.txt
```

## Requirements Versioning {.smaller-90}

You can specify the version of the required packages in the `requirements.txt` file.  This is referred to as "pinning" the versions of the required packages. For example, to require version 2.25.0 of the `requests` package, you can add the following line to the `requirements.txt` file:

```
requests==2.25.0
```

You can also specify a range of versions. For example, to require version 2.25.x of the `requests` package, you can add the following line to the `requirements.txt` file:

```
requests>=2.25.0,<2.26.0
```

## API Data Endpoint {.smaller-75}

The API is described in the [NOAA API documentation](https://www.ncdc.noaa.gov/cdo-web/webservices/v2). We will be using the following data endpoint `https://www.ncei.noaa.gov/cdo-web/api/v2/data` to download the weather data.

We will specify the following parameters in the query string:

- `datasetid=GHCND` (to specify the dataset Global Historical Climatology Network)
- `stationid=GHCND:USW00093814` (to specify the Cincinnati/Northern Kentucky International Airport weather station)
- `datatypeid=TMAX,TMIN`
- `units=standard` (to specify standard units - Fahrenheit)
- `startdate=<start_date>` (to specify the start date)
- `enddate=<end_date>` (to specify the end date)
- `limit=1000`


## The Curse of the Live Demo {.hide}

The curse of the live demo ... appears to have been caused by **hurricane Helene!**

![](./helene.png)

## Token Security {.smaller-90}

You should keep your API token secure. Do not share it with others or post it publicly. You can store your API token in a file named `.env` in the same directory as your code. The `.env` file should contain the following line:

```
NOAA_TOKEN=your-api-token
```

## Test Code {.hide .smaller-55}

Let's make sure your API token is working by running the following code, which will download the weather data for the Cincinnati/Northern Kentucky International Airport weather station for the month of October 2024.

```python
import requests
import os
from dotenv import load_dotenv

load_dotenv()
noaa_token = os.getenv("NOAA_TOKEN")

base_url = "https://www.ncdc.noaa.gov/cdo-web/api/v2"
data_endpoint = f"{base_url}/data"
headers = {"token": noaa_token}

params = {
    "datasetid": "GHCND",
    "stationid": "GHCND:USW00093814",
    "startdate": "2024-10-01",
    "enddate": "2024-10-31",
    "datatypeid": "TMAX,TMIN",  # Only temperature data types
    "units": "standard",  # Use standard units (Fahrenheit)
    "limit": 1000
}

response = requests.get(
    data_endpoint,
    headers=headers,
    params=params
)

response.raise_for_status()

if response.status_code == 200:
    data = response.json()

## Print the first 5 results
data['results'][:5]
```

## Basic Temperature Forecasting {.hide .smaller-85}

:::: {.columns}

::: {.column width=40%}
For a basic temperature forecasting model, you should ideally have at least 3-5 years of historical data to capture seasonal patterns and year-to-year variations effectively. 10+ years is ideal for capturing longer-term patterns and climate trends.
:::

::: {.column width=60%}
![](./usw00093814-station-details.png)
:::

::::


## `dataclass` {.hide .smaller-90}

In Python, there is a special decorator called `dataclass` that can be used to create classes that are primarily used to store data. A `dataclass` automatically generates special methods such as `__init__`, `__repr__`, and `__eq__` for the class.  A `dataclass` is essentially a class that has boilerplate methods generated automatically.

::: {.notes}
- [Data Classes](https://docs.python.org/3/library/dataclasses.html)
A data class is a class typically containing mainly data, without complex logic. Data classes are similar to namedtuples, but are more flexible.  Data classes are created using the `@dataclass` decorator.  Data classes are classes with boilerplate methods generated automatically.
:::

## `noaa.py` {.smaller-80}

Create a new Python file named `noaa.py` and add the following code to define all of the imports that will be used in the file.:

```python
from dataclasses import dataclass
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
import requests
import matplotlib.pyplot as plt
from prophet import Prophet
```

## Step 1 {.hide .smaller-60}

**Step 1**: We will create a `GHCNDStationTemps` dataclass to interact with the NOAA API and retrieve historical temperatures from a station.

::: {.callout-info}
Notice that there is no need to define the `__init__` method explicitly. The `dataclass` decorator automatically generates the `__init__` method for the class.
:::

In [15]:
from dataclasses import dataclass
from datetime import datetime
from datetime import timedelta

@dataclass
class GHCNDStationTemps:
    token: str
    stationid: str
    end_date: datetime = datetime.now().date() - timedelta(days=1)
    years_of_history: int = 3
    endpoint: str = "https://www.ncdc.noaa.gov/cdo-web/api/v2/data"


## Step 1: Testing {.hide .smaller-60}

```python
from noaa import GHCNDStationTemps
from dotenv import load_dotenv
import os

load_dotenv()

token = os.getenv("NOAA_TOKEN")
stationid = "GHCND:USW00014739"
ghcnd = GHCNDStationTemps(token, stationid, years_of_history=3)
```

## Step 2 {.hide .smaller-60}

**Step 2**: We will create a `fetch_data` method in the `GHCNDStationTemps` dataclass to fetch the historical temperature data from the NOAA API.

```python
def fetch_data(self):
    if self.end_date is None:
        self.end_date = datetime.now() - timedelta(days=1)

    start_date = self.end_date - relativedelta(years=self.years_of_history)
    current_date = start_date
    all_data = []

    while current_date < self.end_date:
        chunk_end = min(current_date + relativedelta(years=1) - timedelta(days=1), self.end_date)
        params = {
            "datasetid": "GHCND",
            "stationid": self.stationid,
            "datatypeid": "TMAX,TMIN",
            "units": "standard",
            "startdate": current_date.strftime("%Y-%m-%d"),
            "enddate": chunk_end.strftime("%Y-%m-%d"),
            "limit": 1000,
        }

        print(f"Fetching data from {current_date.date()} to {chunk_end.date()}...")
        response = requests.get(self.endpoint, headers={"token": self.token}, params=params)
        response.raise_for_status()

        data = response.json()
        if "results" in data:
            all_data.extend(data["results"])
            print(f"Retrieved {len(data['results'])} records.")
        else:
            print(f"No data found for {current_date.date()} to {chunk_end.date()}.")

        current_date = chunk_end + timedelta(days=1)

    return all_data
```

## Step 2: Testing {.hide .smaller-60}

```python
from noaa import GHCNDStationTemps
from dotenv import load_dotenv
import os

load_dotenv()

token = os.getenv("NOAA_TOKEN")
stationid = "GHCND:USW00014739"
ghcnd = GHCNDStationTemps(token, stationid, years_of_history=3)

ghcnd.fetch_data()
```

## Lazy Loading

Lazy loading is a technique that delays the initialization of resources or objects until they are needed - this can help to improve performance. In our case, we are going to use lazy loading to fetch the data only when it is needed.

## Step 3 {.hide .smaller-60}

**Step 3**: Let's add a `data` property to the `GHCNDStationTemps` dataclass to return the historical temperature data.  This property will call the `fetch_data` method if the data is not already available.

We will add a private `_data` attribute to the class to store the historical temperature data, initialized to `None`.

```python
_data = None
```

Next, modify the `fetch_data` method to be a private method by adding an underscore `_` to the method name.
```python
def _fetch_data(self):
```

Finally, we will define the `data` property to return the historical temperature data.
```python
@property
def data(self):
    if self._data is None:
        print("Data is empty. Fetching data...")
        self._data = self.fetch_data()
    return self._data
```



## Step 3: Testing {.hide .smaller-60}

```python
from noaa import GHCNDStationTemps
from dotenv import load_dotenv
import os

load_dotenv()

token = os.getenv("NOAA_TOKEN")
stationid = "GHCND:USW00014739"
ghcnd = GHCNDStationTemps(token, stationid, years_of_history=3)

ghcnd.data
```

## Step 4a {.hide .smaller-60}

**Step 4a**: Let's modify `data` to return a `pandas` DataFrame instead of a list of dictionaries. Create a private `_convert_to_dataframe` static method to convert the raw data into a `pandas` DataFrame. 

```python
@staticmethod
def _convert_to_dataframe(all_data):
    """
    Convert raw data into a pandas DataFrame with 'Date', 'TMAX', and 'TMIN' columns.
    """
    # Convert to DataFrame and handle missing keys/values (e.g., NaN)
    df = pd.DataFrame(all_data)

    # Explicitly keep only the required columns
    columns_to_keep = ['date', 'datatype', 'value']
    if not all(col in df.columns for col in columns_to_keep):
        raise ValueError(f"Input data must contain these columns: {columns_to_keep}")
    
    df = df[columns_to_keep]

    # Pivot the table to get 'datatype' values as columns
    df_pivot = df.pivot(index='date', columns='datatype', values='value')
    # Remove the MultiIndex column formatting by renaming or flattening
    df_pivot.columns.name = None
    df_pivot = df_pivot.reset_index()

    # Convert 'date' column to datetime.date type
    df_pivot['date'] = pd.to_datetime(df_pivot['date']).dt.date
    
    # Rename columns for better readability
    df_pivot.rename(columns={'TMAX': 'Max Temperature (°F)', 
                                'TMIN': 'Min Temperature (°F)',
                                'date': 'Date'},
                                inplace=True)

    return df_pivot
```

## Step 4b {.hide .smaller-60}

**Step 4b**: Modify the `data` property to return a `pandas` DataFrame.

```python
@property
def data(self):
    if self._data is None:
        print("Data is empty. Fetching and converting data...")
        raw_data = self._fetch_data()
        self._data = self._convert_to_dataframe(raw_data)
    return self._data
```

## Step 4: Testing {.hide .smaller-60}

```python
from noaa import GHCNDStationTemps
from dotenv import load_dotenv
import os

load_dotenv()

token = os.getenv("NOAA_TOKEN")
stationid = "GHCND:USW00014739"
ghcnd = GHCNDStationTemps(token, stationid, years_of_history=3)

ghcnd.data
```

## Step 5 {.hide .smaller-60}

**Step 5**: Let's add a `plot` method to the `GHCNDStationTemps` dataclass to plot the historical temperature data.

```python
def plot_temperatures(self):
    if self.data is None or self.data.empty:
        print("No data available to plot.")
        return

    plt.figure(figsize=(10, 6))
    plt.plot(self.data['Date'], self.data['Max Temperature (°F)'], label='Max Temperature (°F)', color='red')
    plt.plot(self.data['Date'], self.data['Min Temperature (°F)'], label='Min Temperature (°F)', color='blue')
    plt.xlabel('Date')
    plt.ylabel('Temperature (°F)')
    plt.title(f'Temperature Trends for Station {self.stationid}')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
```

## Step 5: Testing {.hide .smaller-60}

```python
from noaa import GHCNDStationTemps
from dotenv import load_dotenv
import os

load_dotenv()

token = os.getenv("NOAA_TOKEN")
stationid = "GHCND:USW00014739"
ghcnd = GHCNDStationTemps(token, stationid, years_of_history=3)

ghcnd.data

ghcnd.plot_temperatures()
```

## Step 6a {.hide .smaller-60}

We are now ready to model the temperature data. We will use the `prophet` package to fit a time series model to the temperature data.  The `prophet` package is a forecasting tool developed by Facebook, it is described as a a modular regression model with interpretable parameters that can be intuitively adjusted by analysts with **domain knowledge** about the time series.

![](./prophet.png)

:::{.callout-info}
This lecture isn't intended to provide a detailed explanation of time series modeling. For more information on time series modeling, refer to the [Prophet documentation](https://facebook.github.io/prophet/).  It is intended to demonstrate functionality on how a Python application can be built to model weather data.
:::



## Prophet {.smaller-65}

Prophet uses a similar API to `sklearn` for fitting and predicting models. Specifically, you create an instance of the `Prophet` class and call its `fit` and `predict` methods.  

The input to Prophet is a dataframe with two required columns: `ds` and `y`. The `ds` (datestamp) column should contain dates or timestamps in a format recognized by Pandas (e.g., YYYY-MM-DD for dates or YYYY-MM-DD HH:MM:SS for timestamps). The `y` column must be numeric and represents the variable to forecast. Ensure that `ds` contains unique values, as duplicate timestamps will result in errors.  

By default, Prophet uses a linear growth model for its forecast. For scenarios where growth is constrained by a maximum achievable point—such as total market size or population size—Prophet allows you to use a logistic growth model. This requires specifying a carrying capacity, which represents the point at which growth saturates. To use the logistic growth model, add a `cap` column to your dataframe that defines the carrying capacity for each time point.

## Domain Knowledge {.hide .smaller-65}

Do we need to use worry about the carrying capacity for temperature data?  **Probably not.**  The temperature data is unlikely to saturate at a certain point.  However, if you were modeling population growth, you would need to consider the carrying capacity.  Why Growth Models Aren’t Relevant for Temperature Forecasting:

- **Temperature is Cyclical, Not Growth-Driven**
  Temperature data is inherently cyclical (daily, seasonal, yearly patterns) rather than exhibiting sustained growth over time. Unlike datasets like sales or population, where there might be exponential or logistic growth trends, temperatures don’t have a "carrying capacity" or an upward trend that needs to be modeled in this way.

- **Stable Mean Over Time**
  While climate change can lead to long-term trends in temperature (e.g., gradual warming), these trends are slow and typically better modeled as part of a linear trend or external covariate (e.g., CO₂ levels), rather than requiring a growth model.

- **Logistic Growth Doesn't Apply**
  Logistic growth assumes a saturating maximum value, which makes sense for bounded phenomena like market size or biological populations. Temperature fluctuations, however, are bounded by physical realities (e.g., Earth's climate system) but do not "grow" towards a carrying capacity in the same way.


## Seasonality {.smaller-65}

Prophet will by default fit weekly and yearly seasonalities.  Prophet models seasonality using a Fourier series representation --- which approximates periodic patterns (like yearly or weekly seasonality) using sine and cosine functions of different frequencies.  Yearly seasonality will use a default Fourier order of 10, and for weekly seasonality a default Fourier order of 3 is used.

:::{.callout-info}
While temperature exhibits yearly seasonality as its primary cycle (driven by seasons), there can also be sub-seasonal variations within months, especially in regions where temperatures change rapidly (e.g., spring or autumn transitions).
:::

You can add other seasonalities (monthly, quarterly, hourly) using the `add_seasonality` method (Python).
The inputs to this function are a name, the period of the seasonality in days, and the Fourier order for the seasonality. 

```python
model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
```

## Step 6b {.hide .smaller-60}

```python
def forecast_daily_temperatures(self, n_periods=365, add_monthly_seasonality=False, plot_forecast=True):
    if self.data is None or self.data.empty:
        raise ValueError("No data available to forecast.")

    # Prepare data for Prophet
    df = self.data[['Date', 'Max Temperature (°F)']].rename(
        columns={'Date': 'ds', 'Max Temperature (°F)': 'y'})

    # Initialize the Prophet model
    # Prophet will by default fit weekly and yearly seasonalities
    model = Prophet(
        yearly_seasonality=True,  # Enable yearly seasonality
        daily_seasonality=False,  # Disable daily seasonality unless needed
        weekly_seasonality=False,  # Disable weekly seasonality unless needed
    )

    # Add additional seasonality if required (e.g., monthly or weekly)
    if add_monthly_seasonality:
        model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
    
    # Fit the model
    model.fit(df)
    
    # Create a dataframe for future dates
    future = model.make_future_dataframe(periods=n_periods)  # Forecast for speicified number of periods

    # Predict future values
    forecast_df = model.predict(future)

    if plot_forecast:
        model.plot(forecast_df)
        plt.title(f"Temperature Forecast for Station {self.stationid}")
        plt.xlabel("Date")
        plt.ylabel("Temperature (°F)")
        plt.show()

    return forecast_df
```

## Step 6: Testing {.hide .smaller-60}

```python
from noaa import GHCNDStationTemps
from dotenv import load_dotenv
import os

load_dotenv()

token = os.getenv("NOAA_TOKEN")
stationid = "GHCND:USW00014739"
ghcnd = GHCNDStationTemps(token, stationid, years_of_history=3)

ghcnd.data

ghcnd.plot_temperatures()

forecast = ghcnd.forecast_daily_temperatures(n_periods=365, plot_forecast=True)
```
